In [2]:
import re
import json as js
import os
import demoji

In [6]:
# 6/6/24, 9:17 PM - +39 370 703 0730: la gpu
def get_datehour(line):
    # return date and hour object
    return line.split()[0][:-1].replace("/", "-"), line.split()[1][:-1]


In [7]:
def check_valid(line):
    # Regex for check if the line follow the format: 6/6/24, 9:17 PM - +39 370 703 0730: la gpu
    return re.match(r"\d+/\d+/\d+, \d+:\d+\s[AP]M - \+\d+ \d+ \d+: .+", line) is not None

In [3]:
file_name = "sample_whatsapp"
current_date = ""
with open("./exported_chat/" + file_name + ".txt", "r", encoding="utf-8") as file:
    data = file.readlines()
    for line in data:
        # Check if the line is valid
        if not check_valid(line):
            continue
        date, hour = get_datehour(line)
        if current_date == "":
            data = date
        if current_date != date:
            past_date = date
        # Create or append to the file
        with open(file_name + "_" + past_date + ".txt", "a", encoding="utf-8") as new_file:
            new_file.write(line)
# Output



FileNotFoundError: [Errno 2] No such file or directory: './exported_chat/sample_whatsapp.txt'

In [4]:

# Create json file

with open(file_name  + ".txt", "r", encoding="utf-8") as file:
    data = file.readlines()
    json = {}
    json["dataset_name"] = "CustomDATA"
    json["dialogue_type"] = "Open-Domain"
    json["reference_summ"] = ""
    dialogue = []
    speaker = []
    for line in data:
        json_sent = {}
        #if not check_valid(line):
        #    continue
        author = line[17:].split(":")[0]
        if author not in speaker:
            speaker.append(author)
        json_sent["speaker"] = "Speaker" + str(speaker.index(author) + 1)
        print(line)
        json_sent["sentences"] = [line.split(":")[1]]
        json_sent["utterance"] = line.split(":")[1]
        json_sent["role"] = author
        dialogue.append(json_sent)
    json["dialogue"] = dialogue
    print(dialogue)
    with open(file_name + "_" + past_date + ".json", "w", encoding="utf-8") as new_file:
        js.dump(json, new_file)

FileNotFoundError: [Errno 2] No such file or directory: 'sample_whatsapp.txt'

In [6]:
# \d{1,2}/\d{1,2}/\d{2}, \d{1,2}:\d{2} (?:AM|PM) - 
def clean_chat(file_name, keepEmojiText=False):
    text=""
    regex = r"\d{1,2}\/\d{1,2}\/\d{2}, \d{1,2}:\d{2}.(?:AM|PM) - "
    with open(file_name, 'r', encoding="utf-8") as file:
        for line in file:
            if re.match(regex, line):
                line = re.sub(regex, "", line)
            line = re.sub(r"\n", " ", line)
            if not keepEmojiText:
                line = demoji.replace(line)
            else:
                line = demoji.replace_with_desc(line, "|")
            text += line
    return text
        
# print(clean_chat("./exported_chat/sample_whatsapp1.txt", True))         

In [28]:
'''
create a dictionary with the following structure: id, text, golden_summary
'''
def create_dict(whatsapp_file, id, golden_summary_path):
    with open(golden_summary_path, 'r', encoding='utf-8') as file:
        summary = file.read()

    chat = { 
        "text": clean_chat(whatsapp_file),
        "id": id,
        "golden_summary": summary
    }
    return chat

In [29]:
path = ".\exported_chat"
dataset = []
for i, dir in enumerate(os.listdir(path)):
    chat_file_path = os.path.join(path, dir, "sample_whatsapp.txt")
    summary_file_path = os.path.join(path, dir, "summary.txt")
    if os.path.exists(chat_file_path) and os.path.exists(summary_file_path):
        dataset.append(create_dict(chat_file_path, i, summary_file_path))

dataset


[{'text': "12/06/2024, 9:00 AM - Alice: Hey everyone, have you thought about our summer vacation yet?  12/06/2024, 9:02 AM - Bob: Yes! I was thinking about going to the beach. Here's a picture of the place I found. (image)  12/06/2024, 9:05 AM - Charlie: Wow, that looks amazing, Bob! I'm totally in.   12/06/2024, 9:10 AM - Dave: Looks great! I'm just concerned about the budget. How much would it cost us?  12/06/2024, 9:12 AM - Alice: I did some research. It's about $500 each for a week, including accommodation.  12/06/2024, 9:15 AM - Bob: That's not bad. Here's an audio clip of the waves from the beach. It's so relaxing! (audio)  12/06/2024, 9:18 AM - Charlie: That sounds perfect! I can already imagine us there.   12/06/2024, 9:20 AM - Dave: I'm in too. Let's book it!  12/06/2024, 9:22 AM - Alice: Awesome! I'll handle the booking.  12/06/2024, 9:25 AM - Bob: Should we rent a car to explore the area?  12/06/2024, 9:28 AM - Charlie: Definitely. I found a rental service that's affordable.